In [ ]:
import numpy as np
import scipy
import sympy
import matplotlib.pyplot as plt

# Some population models in biology

Let's look at some more nonlinear ODEs so you can get more practice with them.
All of these should be solvable using the approach I showed in the Lorenz notebook.

The simplest models look at the population $p$ of a single species, which follows some ODE.
Of these, the most elementary is the exponential growth model $\dot p = \tau^{-1}p$ for some timescale $\tau$.
Here we'll explore a slightly more complicated model.
Then we can look at models for the populations $p$, $q$ of two interacting species, like prey and predators.

### Growth with carrying capacity

Remember early on in the heady days of the pandemic when everyone was discovering what exponential growth was?
More realistic population models account for the fact that the environment has some *carrying capacity* of organisms.
For example, say we were looking at the growth of cyanobacteria on the surface of a lake, and $p$ represents the number of cells in the water column per unit centimeter squared.
Once the population reaches its carrying capacity $p_c$, the population will fall back down.
Likewise, if we look at the population of some herbivorous species like rabbits or deer, the environment only supports so much available food.
Consequently, we might imagine that the growth rate slows multiplicatively as the population approaches capacity:
$$\dot p = \tau^{-1}p\cdot\left(1 - \frac{p}{p_c}\right).$$
This is called the **logistic equation**.

I found [this paper](https://doi.org/10.1186/s12302-021-00483-1) which showed that some lakes have carrying capacities $p_c$ on the order of 10 mm${}^3$ / L and [this paper](https://doi.org/10.1186/s12934-023-02035-z) which tabulated growth timescales $\tau$ on the order of 12h for some species at optimal conditions.

Write some code in sympy to form the right-hand side of the logistic equation.
Make the timescale and carrying capacity arguments symbolic.

In [ ]:
p = sympy.symbols("p", real=True)
τ, p_c = sympy.symbols("τ p_c", real=True, positive=True)
f = ...

Now write some code to lambdify the expression.
Make sure that the arguments are first $p$, then a numpy array of parameters, just like we did for the Lorenz system.
We want the result to be equivalent to a function that might start like this:
```python
def F(p, params):
    τ = params[0]
    p_c = params[1]
    ...
```

In [ ]:
F = ...

Now define the parameters array.
We'll use units of hours and mm${}^3$/L.
Take $\tau$ = 12 hours and $p_c$ = 10 mm${}^3$/L.

In [ ]:
timescale = 12.0
capacity = 10.0
params = ...

If you lambdified $f$ correctly, the code below should work.

In [ ]:
print(f"Growth rate at half capacity: {F(0.5 * capacity, params):.3f} mm³ / L / hr")

Now compute the derivative of $f$ symbolically and lambdify it.

In [ ]:
dF = ...

The growth rate is maximized at half capacity, so the derivative of $f$ should be 0 when evaluated there.

In [ ]:
dF(0.5 * capacity, params)

Now solve the logistic equation using the backward or midpoint method or a linearly implicit version.
Use a final time of 10 $\times$ the timescale $\tau$, a timestep of $\tau / 100$, and an initial population of $p_c / 100$.
This time, try to make your procedure flexible w.r.t. the timestep.

If you're using scipy.optimize.root, you might get a warning from numpy if you try to assign the next value of the solution to be `result.x`.
If the warning makes your eyes twitch, you can fix this with `result.x[0]`.

In [ ]:
G = ...
dG = ...

In [ ]:
final_time = 10.0 * timescale
dt = timescale / 100.0
num_steps = int(final_time / dt)
p_0 = capacity / 100.0

...

Plot the results.

In [ ]:
...

In class, I'll show the analytical solution and how to do a convergence test.

### Predator-prey

Now let's look at the interaction of two species, a prey species $p$ and a predator species $q$.
The prey species grows exponentially, but declines according to predation.
Likewise, the predator species grows according to how many of the prey species it can eat, but individuals die at some fixed rate.
The frequency of predation is proportional to the product $p\cdot q$ of the two populations.
Putting all this together, we get the system
$$\begin{align}
\dot p & = a_1 p - a_2 pq \\
\dot q & = -b_1 q + b_2pq
\end{align}$$
This is called the *Lotka-Volterra* model.
One of the characteristic features is that it can exhibit limit cycles.
With low predator population, the prey population grows rapidly; the predator population then eats the prey, causing the predator population to grow and the prey to decline; finally, the prey population is insufficient to maintain the predators, and the predators then begin to die off, starting the cycle anew.

The Lotka-Volterra model has some fascinating observational corroboration.
The plot below shows the number of pelts of snowshoe hares and Canadian lynxes sold to the Hudson Bay Company over a period of two decades.
The peaks tend to lag each other.

![hudson bay data](https://jmahaffy.sdsu.edu/courses/f09/math636/lectures/lotka/images/lynxgraph.jpg)

[Joseph Mahaffy](https://jmahaffy.sdsu.edu/courses/f17/math636/beamer/lotvol-04.pdf) estimated the parameters of the model based on these observations and obtained
$$a_1 = 0.453, \quad a_2 = 0.0205, \quad b_1 = 0.790, \quad b_2 = 0.0229$$
together with initial populations of $p = 30$, $q = 4$.

Make a symbolic expression for the right-hand side $f$ of the predator-prey model using sympy.
Now that this is a 2D problem, we'll use a sympy Matrix.

In [ ]:
p, q = sympy.symbols("p q", real=True)
a_1, a_2, b_1, b_2 = sympy.symbols("a_1 a_2 b_1 b_2", real=True, positive=True)

f = sympy.Matrix(
    [
        ...
    ]
)

Now use sympy.solve to compute the equilibria of the system.

In [ ]:
equilibria = ...
equilibria

The code below will generate the Jacobian of the system and symbolically calculate the eigenvalues at each equilibrium.
What do you notice about the equilibria?

In [ ]:
df = f.jacobian((p, q))

for z in equilibria:
    df_z = df.subs({p: z[0], q: z[1]})
    print(list(df_z.eigenvals().keys()))

Here I've done the magic to lambdify $f$ and flatten the result.

In [ ]:
F_ = sympy.lambdify(((p, q), (a_1, a_2, b_1, b_2)), f)
F = lambda *args, **kwargs: F_(*args, **kwargs).flatten()

Make the vector of initial conditions and parameters.

In [ ]:
z_0 = ...
params = ...
F(z_0, params)

Now lambdify the derivative of $f$ and store it in a variable `dF`.

In [ ]:
dF = ...

This code will evaluate the derivative of the right-hand side at the initial conditions just to make sure we don't get an error.

In [ ]:
dF(z_0, params)

Now form the nonlinear system that we'll need to solve the predator-prey equations using the backward method or the midpoint method.

In [ ]:
I = np.eye(2)
G = ...
dG = ...

Take a final time of 40 years and a timestep of 1/80th of a year and make a time series of the result.

In [ ]:
final_time = 40.0
dt = 0.0125
num_steps = int(final_time / dt)

zs = ...

Plot the results below, either showing both populations against time, or each population on a different axis.

In [ ]:
...

If you plot both populations against time, you should see results similar to the data from the Hudson Bay Company above.

There are lots of ways you can alter this model.
For example, if there are no predators at all, the prey population will grow without bound.
You could trying using a carrying capacity for the number of prey, so that their growth rate is instead:
$$\dot p = a_1p(1 - p/p_c) - a_2pq.$$
It's also possible to model multi-species interactions instead of just two.
Later, we'll try an example where there's spatial structure as well.
If you're interested in this sort of thing, [Mark Kot's book](https://doi.org/10.1017/CBO9780511608520) is a fun read.